In [50]:
import pandas as pd
import numpy as np
from data_process import *
from metadata import *
import matplotlib.pyplot as plt
import plotly.express as px
import cufflinks as cf
from plotly.offline import plot, iplot, download_plotlyjs, init_notebook_mode

In [51]:
init_notebook_mode(connected=True)
cf.go_offline()

In [52]:
def toss_win_to_win_count():
    """
    Return the Dataframe with match win to toss win percentage
    """
    toss_match_df = season_summary_df[['toss_won', 'winner']].dropna()
    toss_match_df['win'] = toss_match_df['toss_won'] == toss_match_df['winner']
    toss_match_df['win'] = toss_match_df['win'].astype(int)
    
    toss_match_df['toss'] = 1
    toss_match_df = toss_match_df.groupby(['winner']).sum().reset_index() 
    toss_match_df['toss_win_percentage'] = toss_match_df['win']*100/toss_match_df['toss']
    
    toss_match_df = toss_match_df[toss_match_df['winner'] != 'No Result']
    return toss_match_df
    

toss_win_to_win_count().iplot(kind='bar', x='winner', y='toss_win_percentage', title='Toss Win to Win Percentage',  barmode = 'group')
    

In [53]:
def ground_toss_decision():
    """
    This function will return the dataframe for every ground % of BAT FIRST and BOWL FIRST
    """
    venue_toss_df = season_summary_df[['venue_id','venue_name', 'decision']].dropna()
    venue_toss_df['bat_first'] = venue_toss_df['decision'].str.contains('BAT').astype(int)
    venue_toss_df['toss'] = 1
    venue_toss_df = venue_toss_df.groupby(['venue_id', 'venue_name']).sum().reset_index()
    venue_toss_df['bat_first_percentage'] = venue_toss_df['bat_first']*100/venue_toss_df['toss']
    venue_toss_df['bowl_first_percentage'] = 100 - venue_toss_df['bat_first_percentage']
    return venue_toss_df

ground_toss_decision().iplot(kind='bar', x='venue_name', y=['bat_first_percentage','bowl_first_percentage'], title='Ground Toss Decision', barmode='group')
    
    